SCC0270 - Redes Neurais e Aprendizado Profundo

Alunos:
- 10716504 - Helbert Moreira Pinto
- 10377708 - João Marcos Della Torre Divino

Exercicio 1 - Implementar e treinar o modelo Adaline para reconhecer os símbolos Y e Y invertido (letra “Y” e letra “Y” invertida)

Inicialmente construimos, a partir de matrizes de ordem 5, modelos graficos para os conjuntos de "Y" e de "Y invertido" utilizando o valor 1 para células com conteudo e -1 para representar a falta de conteudo.  
Abaixo vemos imagens dos modelos elaborados.

![Modelos Y](imgs/y.png)   ![Modelos Y Invertido](imgs/y_inv.png)

Para estruturar melhor o projeto, decidimos por criar um gerador de modelos em que podemos parametrizar o tamanho do conjunto gerado bem como o limite maximo de ruido por amostra.  
Note não é problema se quisermos gerar um conjunto maior que o numero de modelos, visto que ao inserirmos um ruido aleatorio as amostras ficam levemente diferenciadas.

```python
gerar_dados(n=100, num_ruido=3)
```
No exemplo acima estamos gerando um conjunto de dados com 100 amostras e com até 3 celulas de ruido por amostra.

O conjunto de dados gerado possui o seguinte formato por amostra:  
- 1 coluna para entrada do bias, sempre com o valor 1
- 25 colunas que representam variaveis explicativas (vetor X) que podem ter o valor 1 ou -1, conforme o modelo e o ruido aplicado
- 1 coluna que representa o label (ou variavel resposta - valor Y), que pode ser 1 para o caso do modelo ser do conjunto "Y" ou -1 para o caso do modelo ser do conjunto "Y invertido"

Na construção do perceptron, utilizamos a orientação a objetos presente na linguagem Python para melhor organizar o codigo.  

![Perceptron](imgs/perceptron.png)

Na figura acima vemos o modelo teorico do perceptron, onde basicamente temos uma saida composta da seguinte equação:  
$$ y_k = \sum_{i=0}^{N} x_{ki} * w_i + \theta_k $$  
onde:  
- $y_k$ é a saida calculada
- $x_k$ é o vetor de variaveis explicativas
- $\theta_k$ é o limiar de ativação (bias)

Na implementação utilzamos como função de ativação o degrau bipolar
```python
# degrau bipolar
def funcao_ativacao(u):
    return -1 if u <= 0 else 1
```


No treino do perceptron, iniciamos os pesos com valores aleatorios, que são ajustados a cada iteração do algoritmo  
```python
def treinar(data):
    # algoritmo so acaba qnd o erro em todas as amostras de treino = 0
    while True:
        epoca += 1
        erro_epoca = 0
        
        for i in data.index:
            # X = variaveis explicativas
            # Y = variavel resposta da instancia
            X = [x for x in data.loc[i, data.columns != 'y']]
            Y = data['y'][i]
            
            # continua enquanto nao prever corretamente a instancia atual
            while True:
                # y = previsao realizada para amostra atual
                # Y = valor esperado
                y = prever(X)
                erro_amostra = comparar(Y, y)
                if erro_amostra == 0:
                    break
                    
                # se valor previsto != esperado, atualiza os pesos
                # delta w = taxa aprendizagem * erro de previsao * vetor de entradas
                delta_w = np.dot((taxa_aprendizagem * erro_amostra), X)
                w += delta_w
                erro_epoca += abs(erro_amostra)
        
        if erro_epoca == 0:
            break
```

A celula abaixo realiza todo o processamento:
- geração do conjunto de dados 
- separação do conjunto entre dados de treino e dados de teste
- treinamento do perceptron
- utiliza os pesos treinados no teste para os testes
- calcula a taxa de erro/acerto (acurácia) na etapa de testes 

In [1]:
# importando classe Perceptron criada no projeto
from src.perceptron import Perceptron
from src.modelos import gerar_dados
from sklearn.model_selection import train_test_split
from random import randint

# função que separa randomicamente os dados em treino/teste
# realiza o processo de aprendizagem com os dados de treino
# calcula a acuracia nos dados de teste
def executar(n_amostras=50, tam_ruido=5, n_testes=100, treino_teste=0.8, taxa_aprend=0.5):
    
    # gera os dados que sera utilizado no processamento
    df = gerar_dados(n=n_amostras, num_ruido=tam_ruido)

    soma_acuracia = 0
    for rodada in range(n_testes):
        # utilizando randomicamente uma seed por rodada
        seed = randint(1, 999999)

        # separando aleatoriamente o conjunto de dados
        df_teste, df_treino = train_test_split(df, test_size=treino_teste, shuffle=True, stratify=df['y'], random_state=seed)

        # treino
        p = Perceptron(n_entradas=len(df.columns)-2, tx=taxa_aprend, seed=seed)
        p.treinar(df_treino)

        # teste
        n = 0
        for i in df_teste.index:
            X = [x for x in df_teste.loc[i, df_teste.columns != 'y']]
            Y = df_teste['y'][i]
            y = p.prever(X)
            n += 1 if Y == y else 0
        
        acuracia = n/len(df_teste)
        soma_acuracia += acuracia

    print('Acuracia Media: {:.2f}%'.format(100*(soma_acuracia/n_testes)))

CASO 1 - Underfit

Quando nosso numero de amostras no conjunto de treino é muito reduzido ou quando as variaveis explicativas não conseguem descrever bem as caracteristicas, temos o caso onde o perceptron não consegue identificar padrões que identificam os distintos conjuntos, e portanto a taxa de acertos no conjunto de teste tende a cair.

In [2]:
# ruido de ate 10 celulas por amostra, ou seja, variavel X explica pouco o conjunto
amostras = 100
ruido = 10 
testes = 500
treino = 0.1
taxa = 1

executar(n_amostras=amostras, tam_ruido=ruido, n_testes=testes, treino_teste=treino, taxa_aprend=taxa)

Acuracia Media: 59.93%


CASO 2 - Overfit

Opostamente ao underfit, quando tentamos ajustar demais os pesos aos dados de treino o algoritmo "decora" a resposta para as entradas, e como no caso anterior, a taxa de acertos tende a cair quando o modelo é exposto aos dados de teste.

In [3]:
# taxa de aprendizagem extremamente baixa, ou seja, os pesos são lentamente ajustados aos dados de treino
amostras = 100
ruido = 5
testes = 500
treino = 0.5
taxa = 0.0001

executar(n_amostras=amostras, tam_ruido=ruido, n_testes=testes, treino_teste=treino, taxa_aprend=taxa)

Acuracia Media: 84.85%


CASO 3 - Ajustado

O modelo ideial é quando os parametros são ajustados para que o modelo identifique as particularidades dos conjuntos sem que esteja sobreajustado à um conjunto especifico. 

In [4]:
amostras = 100
ruido = 5
testes = 500
treino = 0.8
taxa = 0.1

executar(n_amostras=amostras, tam_ruido=ruido, n_testes=testes, treino_teste=treino, taxa_aprend=taxa)

Acuracia Media: 96.60%
